In [1]:
import math
import numpy as np
import json
from copy import deepcopy

In [40]:
def n2letter(n):
    '''0 to 'a', 1 to 'b', ... '''
    return chr(96+n)

def string2duration(string):
    ''' "01:50:19.3177493" to duration in seconds'''
    return 3600*int(string[:2]) + 60*int(string[3:5]) + float(string[6:])

In [65]:
def read_data(path):
    global task_count
    global data
    global tasks
    file = open(path)
    data = json.load(file)
    tasks = data['nodes']
    for task in tasks:
        tasks[task]['Data'] = string2duration(tasks[task]['Data'])
    task_count = len(tasks)
    print("Data loaded successfully. Number of tasks: " + str(task_count))

n_cores = 2
read_data("data/mediumRandom.json")
tasks

Data loaded successfully. Number of tasks: 1000


{'1': {'Data': 2843.6552354, 'Dependencies': []},
 '2': {'Data': 3656.0712509, 'Dependencies': [1]},
 '3': {'Data': 2741.9822376, 'Dependencies': [2, 1]},
 '4': {'Data': 4745.4398667, 'Dependencies': [3, 2, 1]},
 '5': {'Data': 3546.8248903, 'Dependencies': [4, 3, 2, 1]},
 '6': {'Data': 5140.9244211, 'Dependencies': [5, 4, 3, 2, 1]},
 '7': {'Data': 3805.5658098, 'Dependencies': [5, 1]},
 '8': {'Data': 3596.1902359, 'Dependencies': [5, 1, 6, 2]},
 '9': {'Data': 5181.8017436, 'Dependencies': [7, 3, 4, 5, 6]},
 '10': {'Data': 4829.1963826, 'Dependencies': [1, 8, 6, 2, 5]},
 '11': {'Data': 4464.1892297, 'Dependencies': [1, 3, 7, 9, 4, 5]},
 '12': {'Data': 3175.5416896, 'Dependencies': [10, 2, 7, 9, 6]},
 '13': {'Data': 3893.9251895, 'Dependencies': [7, 8, 6, 11, 1, 2, 9]},
 '14': {'Data': 4512.2386242, 'Dependencies': [3, 2, 6, 1]},
 '15': {'Data': 4842.6927442, 'Dependencies': [9, 4, 13, 8, 11]},
 '16': {'Data': 3275.502017, 'Dependencies': [13, 8, 11, 5, 14]},
 '17': {'Data': 3142.3018304

In [67]:
#Tasks to child tasks / Tasks to parents / Task is terminal / Task is inital
task2childs = {task : list() for task in tasks}
task2parents = {task : list() for task in tasks}
for task, info in tasks.items():
    #Add childs
    list_task_parents = info['Dependencies']
    for task_parent in list_task_parents:
        task2childs[str(task_parent)].append(task)
    #Add parents
    task2parents[str(task)] = tasks[task]['Dependencies']
    
def task_is_terminal(task: int):
    return len(task2childs[str(task)]) == 0
def task_is_inital(task: int):
    return len(task2parents[str(task)]) == 0

# print(task2childs)
# print(task2parents)

In [80]:
task2sbl = {}

def save_static_bottom_level(task : int):
    task_duration = tasks[str(task)]["Data"]
    if task_is_terminal(task):
        sbl = task_duration
    else:
        list_sbl_child = list()
        for task_child in task2childs[str(task)]:
            if task_child in task2sbl:
                sbl_child = task2sbl[task_child]
            else:
                sbl_child = save_static_bottom_level(task_child)
            list_sbl_child.append(sbl_child)
        sbl = max(list_sbl_child) + task_duration
                
    task2sbl[task] = sbl
    return sbl

for task in tasks:
    if task_is_inital(task):
        save_static_bottom_level(task)
        
task2sbl



{'997': 3660.8652828,
 '995': 7329.5133886,
 '992': 11101.023038700001,
 '998': 5055.7405084,
 '996': 8493.6840367,
 '999': 3304.9536876,
 '989': 14391.4016881,
 '1000': 3915.3270083,
 '993': 8533.405667500001,
 '977': 18834.8031414,
 '988': 13152.1365696,
 '984': 18431.9208806,
 '991': 11489.718674,
 '970': 21754.6130997,
 '975': 16130.644707700001,
 '990': 11683.1522009,
 '994': 9043.431787900001,
 '979': 16062.4008815,
 '985': 16221.385948699999,
 '972': 19163.564448499998,
 '982': 23733.2820034,
 '980': 28368.5460197,
 '986': 17349.8783016,
 '983': 22095.4240934,
 '973': 32489.8453843,
 '976': 16414.667210400003,
 '981': 13859.817122400002,
 '978': 23739.5876388,
 '960': 37018.2770684,
 '957': 41536.9271517,
 '987': 11862.510626700001,
 '964': 36006.4848702,
 '966': 35854.253736,
 '954': 45775.616077900006,
 '952': 50795.890465300006,
 '967': 26579.3983127,
 '974': 20240.1469126,
 '948': 55087.902937700004,
 '971': 33528.262606,
 '958': 38463.775297299995,
 '937': 59738.1955248,
 '

In [70]:
class Node():

    def __init__(self, parent = None, task_to_add = None, core_where_to_add = None, time_task_start = None, graph = None):
        '''Create a Node object ie a partial scheduling
        parent = parent Node, None if root
        task_to_add : task added to the partial schedule
        core_where_to_add : core where to do task
        time_task_start : instant where the core will start computing the task
        '''
        
        if parent is None:
            self.tasks = graph
            self.schedule = {core : list() for core in range(n_cores)}              #lists of the tasks made on each core, ordered by most recent tasks
            self.cores_available_time = {core : 0 for core in range(n_cores)}       #instant t where core will be available
            self.tasks_done_time = {}                                               #instant t where task was done for each task done
            self.tasks_started_time = {}                                            #instant t where task was started for each task done
                        
        else:
            self.tasks = parent.tasks
            
            self.schedule = deepcopy(parent.schedule)
            self.schedule[core_where_to_add].append(task_to_add)
            
            duration_task = self.tasks[task_to_add]['Data']
            self.cores_available_time = deepcopy(parent.cores_available_time)
            self.cores_available_time[core_where_to_add] = time_task_start + duration_task
            
            self.tasks_started_time = deepcopy(parent.tasks_started_time)
            self.tasks_started_time[task_to_add] = time_task_start 
            
            self.tasks_done_time = deepcopy(parent.tasks_done_time)
            self.tasks_done_time[task_to_add] = time_task_start + duration_task            
    
    def __repr__(self):
        string = ''
        for core_schedule in self.schedule.values():
            for task in core_schedule:
                string += n2letter(int(task))
            string += '|'
        return f"{string[:-1]} --- Cores available at time: {self.cores_available_time}. Tasks started times : {self.tasks_started_time}. Tasks done times : {self.tasks_done_time}"
            
    def is_terminal(self):
        '''Return whether a node is a full schedule'''
        return len(self.tasks_done_time) == task_count
    
    def succ(self):                     
        '''Create and return list of child node of self'''
        childs = list()
        
        #On regarde toutes les tâches qu'on va tenter de rajouter
        for task, info in self.tasks.items():
            if task in self.tasks_done_time: #On passe les taches déjà ajoutées
                continue
            if not all([str(task_required) in self.tasks_done_time for task_required in info['Dependencies']]):   #On ne garde que les taches dont toutes les dépendances ont été réalisées
                continue
            
            time_all_dependencies_done = max([0] + [self.tasks_done_time[str(task_required)] for task_required in self.tasks[task]['Dependencies']])
            
            for core in range(n_cores):
                time_core_available = self.cores_available_time[core]
                time_task_start = max(time_core_available, time_all_dependencies_done)  #We wait that the core end its tasks AND that every dependencies was done.
                child = Node(parent = self, task_to_add=task, core_where_to_add=core, time_task_start=time_task_start)
                childs.append(child)
                
        return childs
        
    def cost(self, child_node):
        '''Return the cost of going from self to child_node, a child node of self
        '''
        g_self = max(self.cores_available_time.values())
        g_child = max(child_node.cores_available_time.values())
        cost = g_child - g_self
        return cost
    
    def compute_g(self):
        return max(self.cores_available_time.values())
    
root = Node(graph = tasks)
a = root.succ()[0]
a.succ()

[ab| --- Cores available at time: {0: 6499.7264863, 1: 0}. Tasks started times : {'1': 0, '2': 2843.6552354}. Tasks done times : {'1': 2843.6552354, '2': 6499.7264863},
 a|b --- Cores available at time: {0: 2843.6552354, 1: 6499.7264863}. Tasks started times : {'1': 0, '2': 2843.6552354}. Tasks done times : {'1': 2843.6552354, '2': 6499.7264863}]

In [73]:
def h(node):
    '''Estimated remaining time of the node-schedule for reaching a terminal node.
    Must understimate true value.'''
    successor_tasks = list()
    for task, info in tasks.items():
        if task in node.tasks_done_time: #On passe les taches déjà ajoutées
            continue
        if not all([str(task_required) in node.tasks_done_time for task_required in info['Dependencies']]):   #On ne garde que les taches dont toutes les dépendances ont été réalisées
            continue
        successor_tasks.append(task)
    if successor_tasks == []:
        return 0
    return max([task2sbl[task] for task in successor_tasks])

h(root), h(a)

(1010068.6656111003, 1007225.0103757003)

In [74]:


class A_star():
    def __init__(self, root):
        self.root = root
    
    def find_best_path(self):
        root.g = 0
        root.f = root.g + h(root)
        self.closed_list = []
        self.open_list = [root]
        while len(self.open_list) > 0:
            next_node_to_explore = self.open_list.pop(0)
            self.closed_list.append(next_node_to_explore)
            if next_node_to_explore.is_terminal():
                print("Result:", next_node_to_explore)
                return 
            for node_child in next_node_to_explore.succ():
                # node_child.g = next_node_to_explore.g + next_node_to_explore.cost(node_child)
                node_child.g = node_child.compute_g() #Other method, doesnt work on all Node classes
                node_child.f = node_child.g + h(node_child)
                self.open_list.append(node_child)
            self.open_list.sort(key = lambda node : node.f)
            
        raise Exception("No path from root to a terminal node")

root = Node(graph=tasks)
a_star = A_star(root = root)
a_star.find_best_path()

KeyboardInterrupt: 